# **Lab1:** Procesamiento de señales 1D (EEG)
En este laboratorio se busca aplicar algunos conceptos de procesamiento de señales 1D para el análisis de señales tomadas en un electroencefalograma (EEG).

In [1]:
# librerías a utilizar
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# esto genera estilos
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
# conectando con drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# moverse a la carpeta donde guardaron el laboratorio
%cd "drive/MyDrive/semillero_DO/1-señales-1D"

/content/drive/MyDrive/semillero_DO/1-señales-1D


In [4]:
!ls

datos  lab1-EEG.ipynb


## 1. Lectura de los datos
La **electroencefalografía** (EEG) es el registro de la actividad eléctrica del cerebro medida en el cuero cabelludo. El dispositivo llamado electroencefalógrafo mide la tensión resultante de los cambios de flujo de corriente iónica dentro de las neuronas del cerebro producidos por
la actividad sináptica cerebral.

Las señales EEG se utilizan para diagnosticar y tratar una variedad de trastornos neurológicos, como la epilepsia, el autismo y la enfermedad de Alzheimer. También se utilizan para investigar el funcionamiento normal del cerebro. Por ejemplo, los científicos utilizan los datos EEG para estudiar cómo el cerebro procesa la información, cómo el cerebro controla los movimientos y cómo el cerebro se adapta al cambio.

### **[Datos disponibles:](https://www.kaggle.com/datasets/nnair25/Alcoholics)**
> Estos datos proceden de un amplio estudio para examinar los correlatos EEG de la predisposición genética al alcoholismo. Contiene mediciones de 64 electrodos colocados en el cuero cabelludo de los sujetos, que fueron muestreados a 256 Hz (3,9 mseg. epoch) durante 1 segundo.

<center>

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/EEG_10-10_system_with_additional_information.svg/512px-EEG_10-10_system_with_additional_information.svg.png)

</center>

---
En el directorio `datos` encuentra 4 carpetas relacionada a 4 pacientes:
- `P1:` paciente con alcoholismo que se expuso a un estímulo.
- `P2:` paciente con alcoholismo que se expuso a dos estímulos diferentes.
- `P3:` paciente con alcoholismo que se expuso a dos estímulos iguales.
- `P4:` paciente de control que se expuso a dos estímulos diferentes.

**Prueba 1:**
Lea, del mismo paciente, un sensor del lado izquierdo y uno del lado derecho. (¿Existe una correlación del comportamiento de las señales?)

**Prueba 2:** Lea de cada paciente el mismo sensor y grafique el comportamiento de las 4 señales. (¿Existe una correlación del comportamiento de las señales?)

## 2. Análisis de datos en Fourier
Vamos a realizar un analisis de frecuencias en el dominio de Fourier para entender la composición de los datos.

> **Nota:** Para realizar este análisis o cualquier filtrado por frecuencias es necesario conocer la frecuencia a la que fueron muestreados los datos ($F_s$).

Teniendo en cuenta que las señales se tomaron en 1 segundo (aproximadamente), ¿cual es la frecuencia de muestreo?.

**Prueba 3:** teniendo en cuenta la frecuencia de muestreo, realice un gráfico para visualizar el aporte de cada frecuencia de muestreo a la señal.

## 3. Limpiando los datos de frecuencias externas

> La señal EEG está sujeta a una gran variedad de artefactos y ruido que muchas veces es difícil de controlar y eliminar. Dentro de los elementos que generan artefactos están: los parpadeos, actividad oculomotora, los movimientos de la cabeza, las expresiones faciales que agregan ruido por la señal eléctrica muscular, movimiento de los electrodos, entre otros.

> Para eliminar el efecto del balanceo de la cabeza se utiliza un filtro pasa alto de frecuencia de corte de $0.5$ Hz. Además, se usa un filtro pasa bajos de frecuencia de corte de $40$ Hz, para elimina ruido de la red eléctrica ($50$ - $60$ Hz).



**Prueba 4:** teniendo en cuenta la información anterior, haga un filtro para eliminar el ruido presente en la señal.

In [18]:
# @markdown Esta celda define una función `filtro_paso()` para hacer un filtro de paso bajo, alto o banda a una señal de entrada.

def filtro_paso(x: np.ndarray, fc: float|list[float], fs: float, tipo: str):
  b, a = signal.butter(5, fc, tipo, fs=fs)
  return signal.filtfilt(b, a, x, method="pad")

## 4. Seleccionando ondas cerebrales
> Hay cinco grandes ondas cerebrales que se distinguen por sus diferentes bandas de frecuencia. Estas bandas de frecuencia se llaman, de baja a alta frecuencia:
- **Delta:** Son las ondas de más baja frecuencia, entre **1 y 4 Hz**. Es decir, son ondas lentas y fuertes. Se generan en la etapa del sueño profundo y reparador. También pueden estar presentes durante la meditación profunda. Nos ayudan a sentirnos regenerados y promueven el sistema inmunológico. <br>
En casos de lesiones cerebrales, problemas de aprendizaje o TDAH severo, las ondas delta pueden observarse de manera demasiado prominente.
- **Theta:** Su frecuencia oscila entre **4 y 8 Hz**. Se alcanzan en estados de calma profunda, cuando nuestros sentidos se enfocan en nuestro mundo interior. La persona que está soñando despierta se encuentra en este estado, así como las personas que meditan. Son las ondas que usamos durante la fase R.E.M del sueño y están muy ligadas al aprendizaje, la memoria y la intuición.
- **Alfa:** Alpha representa un estado de escasa actividad cerebral y relajación. Estas ondas son más lentas y de mayor amplitud que las beta. Su frecuencia oscila entre **8 y 12 Hz**. Una persona que ha terminado una tarea y se sienta a descansar, se encuentra a menudo en un estado alpha. <br>
Estas ondas ayudan a la calma mental, la integración de mente y cuerpo y el aprendizaje. Puede considerarse un puente del mundo exterior al mundo interior, y viceversa. <br>
En pacientes que han experimentado un trauma importante o personas con adicciones, las amplitudes alpha pueden ser bajas y esto lleva a una rigidez y a una evitación al mundo interno.
- **Beta:** Se producen cuando el cerebro está implicado en actividades mentales. Son las de mayor velocidad de transmisión de entre las cuatro. Su frecuencia oscila entre los **12 y 25 Hz**, y denotan una actividad mental intensa. Cuando una persona está ordenando una tarea a su cerebro se encuentra emitiendo este tipo de ondas. <br>
El predominio de ondas beta puede indicar un estado en el que la persona está enfocada en el mundo externo.
- **Gamma:** Este tipo de ondas aparecen cuando estamos en un momento de extrema atención y concentración o bien en momentos altamente estresantes. Son las ondas de máxima frecuencia, por encima de **25 Hz**.

**Prueba 5:** teniendo en cuenta la información anterior, obtenga de la señal las ondas delta ($\delta$), theta ($\theta$), alpha ($\alpha$), beta ($\beta$) y gamma ($\gamma$).

## 5. Identificando carga cognitiva
> La señal EEG sirve para predecir si
una tarea era fácil o difícil para el usuario, muchos estudios han encontrado una baja en la actividad de la banda Alfa junto con, la mayoría de las veces, un aumento en la actividad de la banda Theta va acompañado de un incremento en la demanda atencional y en la carga de trabajo de la memoria. <br>
La relación $$\frac{\beta}{\alpha + \theta}$$ basado en la evidencia de que
aumenta Beta con la ejecución de la tarea, Theta se suprime y Alfa se bloquea.

Dada la información anterior, muestre un estimado de la carga cognitiva del paciente.

¿En que tiempos se dio los mayores valores de carga cognitiva en la persona?